In [1]:
# Importation des modules

import pandas as pd
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim

logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

from gensim.models.phrases import Phrases, Phraser

import multiprocessing

from gensim.models import Word2Vec

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
 
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

#Affichage de toutes les colonnes

pd.set_option('display.max_columns', 500)

In [2]:
df0 = pd.read_csv('data/data_cleaned_NLP.csv', sep = ',', encoding = 'latin-1')

In [3]:
df1 = df0[['Réplique', 'Groupe', 'tokenized_replique']].copy()

df = df1[['Groupe', 'tokenized_replique']].copy()

In [4]:
df = df.dropna()
df.isnull().sum()

Groupe                0
tokenized_replique    0
dtype: int64

In [5]:
sent = [row.split() for row in df['tokenized_replique']]

In [7]:
phrases = Phrases(sent, min_count = 30, progress_per = 10000)

# min_count : Ignore all words and bigrams with total collected count lower than this value.

INFO - 21:25:45: collecting all words and their counts
INFO - 21:25:45: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 21:25:46: PROGRESS: at sentence #10000, processed 289462 words and 218376 word types
INFO - 21:25:47: PROGRESS: at sentence #20000, processed 541314 words and 376873 word types
INFO - 21:25:48: PROGRESS: at sentence #30000, processed 809107 words and 528669 word types
INFO - 21:25:48: PROGRESS: at sentence #40000, processed 1085336 words and 672663 word types
INFO - 21:25:49: PROGRESS: at sentence #50000, processed 1340891 words and 796499 word types
INFO - 21:25:50: PROGRESS: at sentence #60000, processed 1565678 words and 900147 word types
INFO - 21:25:50: PROGRESS: at sentence #70000, processed 1813551 words and 1010525 word types
INFO - 21:25:51: PROGRESS: at sentence #80000, processed 2028492 words and 1100387 word types
INFO - 21:25:51: PROGRESS: at sentence #90000, processed 2303527 words and 1217601 word types
INFO - 21:25:52: PROGRESS: at 

In [8]:
bigram = Phraser(phrases)

INFO - 21:26:13: source_vocab length 4165308
INFO - 21:27:06: Phraser built with 9539 phrasegrams


In [9]:
sentences = bigram[sent]

In [10]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

91992

In [12]:
cores = multiprocessing.cpu_count()

In [13]:
w2v_model = Word2Vec(min_count = 30,
                     window = 2,
                     size = 300,
                     sample = 6e-5, 
                     alpha = 0.03, 
                     min_alpha = 0.0007, 
                     negative = 20,
                     workers = cores - 1)

In [14]:
t = time()

w2v_model.build_vocab(sentences, progress_per = 10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 21:27:43: collecting all words and their counts
INFO - 21:27:43: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 21:27:44: PROGRESS: at sentence #10000, processed 251462 words, keeping 26542 word types
INFO - 21:27:45: PROGRESS: at sentence #20000, processed 472392 words, keeping 35446 word types
INFO - 21:27:46: PROGRESS: at sentence #30000, processed 706898 words, keeping 41337 word types
INFO - 21:27:47: PROGRESS: at sentence #40000, processed 948477 words, keeping 45772 word types
INFO - 21:27:48: PROGRESS: at sentence #50000, processed 1171159 words, keeping 48836 word types
INFO - 21:27:49: PROGRESS: at sentence #60000, processed 1366810 words, keeping 51338 word types
INFO - 21:27:50: PROGRESS: at sentence #70000, processed 1582963 words, keeping 53473 word types
INFO - 21:27:51: PROGRESS: at sentence #80000, processed 1769282 words, keeping 55319 word types
INFO - 21:27:52: PROGRESS: at sentence #90000, processed 2008284 words, keeping 57474 word

Time to build vocab: 0.82 mins


In [15]:
t = time()

w2v_model.train(sentences, total_examples = w2v_model.corpus_count, epochs = 10, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 21:28:32: training model with 3 workers on 26575 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2
INFO - 21:28:33: EPOCH 1 - PROGRESS: at 0.90% examples, 77723 words/s, in_qsize 0, out_qsize 0
INFO - 21:28:34: EPOCH 1 - PROGRESS: at 2.30% examples, 96404 words/s, in_qsize 0, out_qsize 0
INFO - 21:28:35: EPOCH 1 - PROGRESS: at 4.07% examples, 105396 words/s, in_qsize 0, out_qsize 0
INFO - 21:28:36: EPOCH 1 - PROGRESS: at 5.58% examples, 108164 words/s, in_qsize 0, out_qsize 0
INFO - 21:28:38: EPOCH 1 - PROGRESS: at 6.91% examples, 106146 words/s, in_qsize 0, out_qsize 0
INFO - 21:28:39: EPOCH 1 - PROGRESS: at 7.52% examples, 96292 words/s, in_qsize 0, out_qsize 0
INFO - 21:28:40: EPOCH 1 - PROGRESS: at 8.67% examples, 95809 words/s, in_qsize 1, out_qsize 0
INFO - 21:28:41: EPOCH 1 - PROGRESS: at 10.24% examples, 98085 words/s, in_qsize 0, out_qsize 0
INFO - 21:28:42: EPOCH 1 - PROGRESS: at 12.21% examples, 100982 words/s, in_qsize 1, out_qsize 0
INFO

INFO - 21:29:57: EPOCH 2 - PROGRESS: at 23.44% examples, 100187 words/s, in_qsize 0, out_qsize 0
INFO - 21:29:58: EPOCH 2 - PROGRESS: at 24.76% examples, 100204 words/s, in_qsize 1, out_qsize 0
INFO - 21:29:59: EPOCH 2 - PROGRESS: at 25.88% examples, 99513 words/s, in_qsize 0, out_qsize 0
INFO - 21:30:00: EPOCH 2 - PROGRESS: at 26.91% examples, 99021 words/s, in_qsize 0, out_qsize 0
INFO - 21:30:01: EPOCH 2 - PROGRESS: at 27.82% examples, 97827 words/s, in_qsize 0, out_qsize 0
INFO - 21:30:02: EPOCH 2 - PROGRESS: at 29.32% examples, 97706 words/s, in_qsize 0, out_qsize 0
INFO - 21:30:03: EPOCH 2 - PROGRESS: at 31.02% examples, 97559 words/s, in_qsize 0, out_qsize 0
INFO - 21:30:04: EPOCH 2 - PROGRESS: at 32.73% examples, 97355 words/s, in_qsize 0, out_qsize 0
INFO - 21:30:06: EPOCH 2 - PROGRESS: at 34.84% examples, 98438 words/s, in_qsize 0, out_qsize 0
INFO - 21:30:07: EPOCH 2 - PROGRESS: at 39.15% examples, 98678 words/s, in_qsize 0, out_qsize 0
INFO - 21:30:08: EPOCH 2 - PROGRESS: a

INFO - 21:31:26: EPOCH 3 - PROGRESS: at 27.99% examples, 101318 words/s, in_qsize 0, out_qsize 0
INFO - 21:31:27: EPOCH 3 - PROGRESS: at 29.55% examples, 101509 words/s, in_qsize 0, out_qsize 0
INFO - 21:31:28: EPOCH 3 - PROGRESS: at 31.65% examples, 101313 words/s, in_qsize 0, out_qsize 0
INFO - 21:31:29: EPOCH 3 - PROGRESS: at 33.19% examples, 101423 words/s, in_qsize 0, out_qsize 0
INFO - 21:31:30: EPOCH 3 - PROGRESS: at 35.43% examples, 101374 words/s, in_qsize 0, out_qsize 0
INFO - 21:31:31: EPOCH 3 - PROGRESS: at 39.06% examples, 100742 words/s, in_qsize 0, out_qsize 0
INFO - 21:31:32: EPOCH 3 - PROGRESS: at 40.09% examples, 99831 words/s, in_qsize 0, out_qsize 0
INFO - 21:31:33: EPOCH 3 - PROGRESS: at 41.16% examples, 99251 words/s, in_qsize 0, out_qsize 0
INFO - 21:31:35: EPOCH 3 - PROGRESS: at 42.45% examples, 98864 words/s, in_qsize 0, out_qsize 0
INFO - 21:31:36: EPOCH 3 - PROGRESS: at 43.63% examples, 98320 words/s, in_qsize 0, out_qsize 0
INFO - 21:31:37: EPOCH 3 - PROGRES

INFO - 21:32:53: EPOCH 4 - PROGRESS: at 54.96% examples, 119957 words/s, in_qsize 0, out_qsize 0
INFO - 21:32:54: EPOCH 4 - PROGRESS: at 56.98% examples, 120288 words/s, in_qsize 0, out_qsize 0
INFO - 21:32:55: EPOCH 4 - PROGRESS: at 58.66% examples, 120437 words/s, in_qsize 0, out_qsize 0
INFO - 21:32:56: EPOCH 4 - PROGRESS: at 60.48% examples, 120749 words/s, in_qsize 0, out_qsize 0
INFO - 21:32:57: EPOCH 4 - PROGRESS: at 62.20% examples, 121044 words/s, in_qsize 0, out_qsize 0
INFO - 21:32:58: EPOCH 4 - PROGRESS: at 64.25% examples, 121313 words/s, in_qsize 1, out_qsize 0
INFO - 21:32:59: EPOCH 4 - PROGRESS: at 65.54% examples, 120952 words/s, in_qsize 0, out_qsize 0
INFO - 21:33:00: EPOCH 4 - PROGRESS: at 67.10% examples, 120956 words/s, in_qsize 0, out_qsize 0
INFO - 21:33:01: EPOCH 4 - PROGRESS: at 68.68% examples, 121268 words/s, in_qsize 1, out_qsize 0
INFO - 21:33:02: EPOCH 4 - PROGRESS: at 70.41% examples, 121432 words/s, in_qsize 1, out_qsize 0
INFO - 21:33:03: EPOCH 4 - PRO

INFO - 21:34:19: EPOCH 5 - PROGRESS: at 74.73% examples, 90310 words/s, in_qsize 0, out_qsize 0
INFO - 21:34:20: EPOCH 5 - PROGRESS: at 75.84% examples, 90068 words/s, in_qsize 0, out_qsize 0
INFO - 21:34:21: EPOCH 5 - PROGRESS: at 78.00% examples, 90338 words/s, in_qsize 0, out_qsize 0
INFO - 21:34:22: EPOCH 5 - PROGRESS: at 79.15% examples, 90369 words/s, in_qsize 0, out_qsize 0
INFO - 21:34:23: EPOCH 5 - PROGRESS: at 80.35% examples, 90323 words/s, in_qsize 0, out_qsize 0
INFO - 21:34:24: EPOCH 5 - PROGRESS: at 84.30% examples, 90141 words/s, in_qsize 0, out_qsize 0
INFO - 21:34:25: EPOCH 5 - PROGRESS: at 85.54% examples, 90198 words/s, in_qsize 1, out_qsize 0
INFO - 21:34:26: EPOCH 5 - PROGRESS: at 86.81% examples, 90235 words/s, in_qsize 0, out_qsize 0
INFO - 21:34:27: EPOCH 5 - PROGRESS: at 88.32% examples, 90389 words/s, in_qsize 1, out_qsize 0
INFO - 21:34:28: EPOCH 5 - PROGRESS: at 89.49% examples, 90331 words/s, in_qsize 0, out_qsize 0
INFO - 21:34:30: EPOCH 5 - PROGRESS: at 

INFO - 21:35:42: EPOCH 7 - PROGRESS: at 10.08% examples, 111355 words/s, in_qsize 1, out_qsize 0
INFO - 21:35:43: EPOCH 7 - PROGRESS: at 12.03% examples, 113060 words/s, in_qsize 1, out_qsize 0
INFO - 21:35:44: EPOCH 7 - PROGRESS: at 13.87% examples, 115011 words/s, in_qsize 0, out_qsize 0
INFO - 21:35:45: EPOCH 7 - PROGRESS: at 15.99% examples, 116615 words/s, in_qsize 0, out_qsize 0
INFO - 21:35:46: EPOCH 7 - PROGRESS: at 17.79% examples, 118156 words/s, in_qsize 1, out_qsize 0
INFO - 21:35:47: EPOCH 7 - PROGRESS: at 19.51% examples, 119348 words/s, in_qsize 1, out_qsize 0
INFO - 21:35:48: EPOCH 7 - PROGRESS: at 21.12% examples, 119828 words/s, in_qsize 1, out_qsize 0
INFO - 21:35:49: EPOCH 7 - PROGRESS: at 23.08% examples, 120656 words/s, in_qsize 0, out_qsize 0
INFO - 21:35:50: EPOCH 7 - PROGRESS: at 24.65% examples, 121093 words/s, in_qsize 1, out_qsize 0
INFO - 21:35:51: EPOCH 7 - PROGRESS: at 26.29% examples, 121831 words/s, in_qsize 1, out_qsize 0
INFO - 21:35:52: EPOCH 7 - PRO

INFO - 21:37:07: EPOCH 8 - PROGRESS: at 34.14% examples, 96235 words/s, in_qsize 0, out_qsize 0
INFO - 21:37:08: EPOCH 8 - PROGRESS: at 38.47% examples, 95813 words/s, in_qsize 0, out_qsize 0
INFO - 21:37:09: EPOCH 8 - PROGRESS: at 39.78% examples, 95825 words/s, in_qsize 0, out_qsize 0
INFO - 21:37:10: EPOCH 8 - PROGRESS: at 41.08% examples, 96083 words/s, in_qsize 0, out_qsize 0
INFO - 21:37:11: EPOCH 8 - PROGRESS: at 42.66% examples, 96659 words/s, in_qsize 0, out_qsize 0
INFO - 21:37:13: EPOCH 8 - PROGRESS: at 44.29% examples, 97462 words/s, in_qsize 0, out_qsize 0
INFO - 21:37:14: EPOCH 8 - PROGRESS: at 45.70% examples, 97737 words/s, in_qsize 1, out_qsize 0
INFO - 21:37:15: EPOCH 8 - PROGRESS: at 47.08% examples, 98182 words/s, in_qsize 0, out_qsize 0
INFO - 21:37:16: EPOCH 8 - PROGRESS: at 48.58% examples, 98338 words/s, in_qsize 0, out_qsize 0
INFO - 21:37:17: EPOCH 8 - PROGRESS: at 50.07% examples, 98722 words/s, in_qsize 0, out_qsize 0
INFO - 21:37:18: EPOCH 8 - PROGRESS: at 

INFO - 21:38:35: EPOCH 9 - PROGRESS: at 62.45% examples, 86390 words/s, in_qsize 0, out_qsize 0
INFO - 21:38:36: EPOCH 9 - PROGRESS: at 63.28% examples, 85503 words/s, in_qsize 0, out_qsize 0
INFO - 21:38:37: EPOCH 9 - PROGRESS: at 64.07% examples, 84798 words/s, in_qsize 0, out_qsize 0
INFO - 21:38:38: EPOCH 9 - PROGRESS: at 64.89% examples, 84241 words/s, in_qsize 0, out_qsize 0
INFO - 21:38:39: EPOCH 9 - PROGRESS: at 65.63% examples, 83793 words/s, in_qsize 0, out_qsize 0
INFO - 21:38:40: EPOCH 9 - PROGRESS: at 66.37% examples, 83345 words/s, in_qsize 0, out_qsize 0
INFO - 21:38:42: EPOCH 9 - PROGRESS: at 67.17% examples, 82841 words/s, in_qsize 0, out_qsize 0
INFO - 21:38:43: EPOCH 9 - PROGRESS: at 67.95% examples, 82459 words/s, in_qsize 0, out_qsize 0
INFO - 21:38:44: EPOCH 9 - PROGRESS: at 68.77% examples, 82192 words/s, in_qsize 0, out_qsize 0
INFO - 21:38:45: EPOCH 9 - PROGRESS: at 69.53% examples, 81704 words/s, in_qsize 0, out_qsize 0
INFO - 21:38:46: EPOCH 9 - PROGRESS: at 

INFO - 21:40:01: EPOCH 10 - PROGRESS: at 77.84% examples, 96305 words/s, in_qsize 1, out_qsize 0
INFO - 21:40:02: EPOCH 10 - PROGRESS: at 79.47% examples, 96756 words/s, in_qsize 0, out_qsize 0
INFO - 21:40:03: EPOCH 10 - PROGRESS: at 82.85% examples, 97091 words/s, in_qsize 0, out_qsize 0
INFO - 21:40:04: EPOCH 10 - PROGRESS: at 85.46% examples, 97335 words/s, in_qsize 0, out_qsize 0
INFO - 21:40:05: EPOCH 10 - PROGRESS: at 87.16% examples, 97584 words/s, in_qsize 0, out_qsize 1
INFO - 21:40:06: EPOCH 10 - PROGRESS: at 89.01% examples, 98166 words/s, in_qsize 0, out_qsize 0
INFO - 21:40:07: EPOCH 10 - PROGRESS: at 90.54% examples, 98649 words/s, in_qsize 0, out_qsize 0
INFO - 21:40:08: EPOCH 10 - PROGRESS: at 92.83% examples, 99132 words/s, in_qsize 0, out_qsize 0
INFO - 21:40:09: EPOCH 10 - PROGRESS: at 94.42% examples, 99229 words/s, in_qsize 0, out_qsize 0
INFO - 21:40:11: EPOCH 10 - PROGRESS: at 95.92% examples, 99178 words/s, in_qsize 0, out_qsize 0
INFO - 21:40:12: EPOCH 10 - PR

Time to train the model: 11.7 mins


In [16]:
w2v_model.init_sims(replace = True)

INFO - 21:40:14: precomputing L2-norms of word weight vectors


In [17]:
#w2v_model.wv.most_similar(positive=["macron"])
#w2v_model.wv.most_similar(negative=["promesse"])
#

[('candidat_emmanuel', 0.6341961026191711),
 ('emmanuel_macron', 0.5966997742652893),
 ('sarkozy', 0.5951269865036011),
 ('candidat_macron', 0.5950851440429688),
 ('nicolas_sarkozy', 0.5803539156913757),
 ('françois_hollande', 0.5756646394729614),
 ('hollande', 0.567308783531189),
 ('président_macron', 0.5493296384811401),
 ('jacques_chirac', 0.5455673933029175),
 ('fillon', 0.5433439016342163)]

0.35283846

In [21]:
w2v_model.wv.similarity("sport", 'études')

0.11891114

In [22]:
print(w2v_model.wv.similarity("macron", 'droite'))
print(w2v_model.wv.similarity("macron", 'gauche'))

0.22307199
0.1712402


In [23]:
w2v_model.wv.doesnt_match(['gauche', 'président', 'droite'])

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'président'

In [24]:
w2v_model.wv.most_similar(positive=["père", "femme"], negative = ['homme'], topn=3)

[('mère', 0.5867539644241333),
 ('maman', 0.5529024600982666),
 ('parent', 0.5426760315895081)]

In [26]:
w2v_model.wv.most_similar(positive=["macron"])

[('candidat_emmanuel', 0.6341961026191711),
 ('emmanuel_macron', 0.5966997742652893),
 ('sarkozy', 0.5951269865036011),
 ('candidat_macron', 0.5950851440429688),
 ('nicolas_sarkozy', 0.5803539156913757),
 ('françois_hollande', 0.5756646394729614),
 ('hollande', 0.567308783531189),
 ('président_macron', 0.5493296384811401),
 ('jacques_chirac', 0.5455673933029175),
 ('fillon', 0.5433439016342163)]

In [31]:
w2v_model['macron']

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


300

In [36]:
?w2v_model

In [72]:
df_export = pd.DataFrame(w2v_model.wv.vectors)

In [73]:
df_export

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.022135,0.053026,0.067560,0.023627,-0.061066,-0.047726,0.104033,0.043060,-0.019778,0.051943,0.055961,-0.051905,-0.031144,0.073373,-0.093625,-0.113178,0.013809,-0.016950,0.045931,-0.051945,-0.108175,0.009362,0.127827,0.011231,-0.057974,0.048334,0.065933,-0.025933,0.079487,0.018273,-0.001864,-0.149220,-0.084360,0.044829,-0.010834,0.053521,0.002652,-0.063327,0.070202,0.037629,-0.015725,0.031606,0.101637,0.010464,0.040698,-0.036039,0.039623,-0.045930,0.023611,0.062255,-0.000478,0.055509,0.058756,-0.086740,-0.019650,0.065463,0.036605,0.039923,-0.037229,-0.027871,0.003588,-0.013307,-0.039206,-0.029150,-0.084981,0.093628,-0.113541,-0.009804,0.068996,-0.019459,-0.048861,-0.017107,0.028295,-0.139427,-0.058019,-0.006566,-0.018306,-0.050091,0.037409,0.005880,-0.059570,0.004537,0.024508,0.004220,-0.058576,-0.022894,0.014006,0.049217,0.045701,0.057634,0.055513,-0.110122,0.102888,0.050329,0.045562,-0.021240,-0.079081,0.024615,-0.028317,0.012814,0.068708,-0.042491,0.014049,-0.020904,0.097639,0.032722,0.017364,-0.001308,0.003148,0.089833,0.114990,-0.077366,0.032009,0.046860,0.027412,0.117288,0.006389,0.020076,0.030676,0.040888,-0.065948,0.101382,-0.032742,0.010638,0.022439,-0.019941,0.049147,0.126401,0.085920,-0.061261,0.016460,-0.031792,0.026847,0.026911,0.007014,0.008214,-0.000080,0.067532,0.052519,0.019863,0.039061,-0.021422,-0.082304,0.040216,-0.104537,-0.057209,-0.024215,-0.080122,-0.001856,-0.026078,0.038998,-0.056474,-0.027262,0.013451,-0.055832,-0.039027,0.122671,-0.064964,-0.043746,-0.103806,0.050550,0.028857,0.082904,0.118245,0.041402,-0.036888,-0.074924,0.024025,0.009609,-0.022330,-0.015446,0.001805,-0.050896,-0.004627,0.028537,-0.096996,-0.004157,0.073544,-0.010703,-0.021927,0.010981,-0.038889,-0.102848,0.131630,0.045923,-0.009582,-0.081338,0.007207,0.108985,-0.173236,0.058692,-0.063246,-0.106067,0.044382,0.106649,0.026572,-0.007799,0.060285,-0.062057,-0.004396,0.038812,-0.027083,0.030060,-0.013969,-0.069897,-0.025844,0.092548,-0.004544,-0.117429,0.048496,-0.052251,-0.141246,-0.066181,-0.001072,0.025048,0.092864,-0.056598,0.030733,-0.048045,0.039058,-0.033325,0.026709,0.070498,0.044074,0.047885,0.015351,0.030606,-0.000671,-0.004314,-0.037853,0.002184,0.133110,-0.058290,-0.031114,-0.023699,-0.033025,-0.030929,-0.029224,-0.050463,0.038782,0.047838,0.001931,-0.048035,-0.066231,0.030654,-0.021602,-0.023354,-0.096962,-0.096845,0.030371,0.049973,0.049506,-0.038750,0.101393,0.015877,0.034413,0.046496,-0.001417,-0.003057,0.059081,-0.054720,-0.088109,-0.041155,-0.041886,0.063401,-0.002016,0.008546,-0.010544,0.128587,0.023280,-0.039994,0.088386,-0.020779,-0.031100,-0.020071,0.026628,-0.052246,-0.005405,-0.055259,0.029182,-0.073301,0.082242,-0.078880,-0.116323,-0.035735,-0.041900,-0.066551,-0.079596,-0.011393,0.047408,-0.027247,0.048641,-0.014351,0.035844,-0.082953,0.022258,0.086866,0.024581,-0.034888,0.036099
1,0.009981,0.032557,0.094099,-0.066324,0.121093,0.033446,-

In [64]:
sorted(list(w2v_model.wv.vocab))[1]

'a_augmenté'

In [83]:
words = [w2v_model.wv.most_similar(positive=[np.array(df_export.iloc[i])])[0][0] for i in range(df_export.shape[0])]

In [84]:
df_export['word'] = words

In [87]:
df_export.to_csv('results/embedding_test.csv')